그리디: 매번 최적의 부분 해 선택
- 최적화(최대/최소) 문제에서 사용
    - 다양한 선택 기준 O
- 항상 최적의 해를 도출할 수 없음 (근사해, 휴리스틱 알고리즘)
    - 반례를 통해 확인 가능
    - 다만 다음 두 조건을 만족하면 항상 최적의 해를 도출할 수 있음
        - 최적의 하위 구조 특성: 최적의 최종 해가 최적의(단 하나의) 부분 해들로 이루어져 있을 때
        - 탐욕적 선택 특성: 선택시 최적의 부분 해만 선택하게 되어 최적의 최종 해를 만들 수 있을 때
            - 만족 못하면 대신 동적 프로그래밍 사용
- 설계 과정
    1. 선정 과정: 최적이라 생각되는 부분 해 선택
    2. 적정성 점검: 선택한 부분 해를 해 집합에 포함시키는 것이 적절한지 확인 후 포함
    3. 해답 점검(종료조건): 선정 과정과 적정성 점검을 반복하면서 새로운 해 집합이 최종 해인지 확인

In [1]:
def coin_change(x):
    d = [1, 5, 10, 50, 100, 500]
    result = []
    i = len(d) - 1 # last index # selection procedure
    while True:
        while x >= d[i]: # feasibility check
            x = x - d[i]
            result.append(d[i])
        i -= 1
        if i < 0: # solution check
            break
    for i in range(len(result)):
        print(result[i], end=" ")

x = 16
coin_change(x)

10 5 1 

신장 트리: 싸이클이 없는(트리) 무방향,가중치,연결 그래프
- 정점: N
- 간선: N-1 (싸이클 X)

최소 신장 트리: 가중치 합이 최소인 신장 트리
- (최소이므로) 그리디 알고리즘으로 찾음
    - Prim 알고리즘
    - Kruskal 알고리즘
- 하나 이상 존재 가능

Kruskal 알고리즘: 신장 트리의 간선(E)를 기반으로 동작, **O(MlogM), O(NlogN)(희소 그래프, 신장 트리) ~ O(N<sup>2</sup>logN)(밀집 그래프)**

- 신장 트리 간선을 가중치 기준 오름차순 정렬(**O(MlogM)**)
- 신장 트리 정점으로 서로소 집합 생성(**O(N)**)
1. 선정 과정: **가중치가 가장 작은** 신장 트리의 **간선 선택**
2. 적정성 검사: 두 정점이 다른 루트(집합)이면(**싸이클을 생성되지 않는다면**) 두 트리(집합) union 후 **최소 신장 트리의 간선에 추가**(**O(MlogM)**)
3. 해답 점검: 정점들이 하나의 집합이 될 때(**최소 신장 트리의 간선이 N-1개일 때**) **종료**

서로소 집합: 최소 신장 트리에서 싸이클 생성 여부 판단할 수 있는 자료구조
- 논리적: 트리
- 물리적: 파이썬 리스트
    - 크기: 노드 수
    - 인덱스: 노드
    - 값: 루트(각 집합의 대표값)/부모
- 연산: find & union
    - find: 루트(해당 집합의 대표값) 반환, **O(MlogM)**
        - 경로 압축을 통해 더 빠른 수행 가능
    - union: 합집합

In [17]:
N = 7 # number of vertexes
weights = [
    (0, 1, 9), (0, 2, 10), (1, 3, 10), (1, 4, 5),
    (1, 6, 3), (2, 3, 9), (2, 4, 7), (2, 5, 2),
    (3, 5, 4), (3, 6, 8), (4, 6, 1), (5, 6, 6)
] # (vertex 1, vertex 2, weight)

weights.sort(key=lambda t: t[2]) # sort ascending

mst = [] # mst edges
p = [] # disjoint sets

for i in range(N):
    p.append(i)

# O(h)(normally O(logN) if biased O(N))
def find(u): # u: node
    if u != p[u]: # p: disjoint sets/python list
        p[u] = find(p[u]) # path compression(change the value to root node)
    return p[u]    

def union(u, v):
    root1 = find(u)
    root2 = find(v)
    p[root2] = root1 # unify under root1

tree_edges = 0
mst_cost = 0

while True:
    if tree_edges == N - 1: # if mst # solution check
        break
    u, v, wt = weights.pop(0) # return first element # selection procedure
    if find(u) != find(v): # if different set # feasibility check
        union(u, v)
        mst.append((u, v))
        mst_cost += wt
        tree_edges += 1

print('최소신장트리:', mst)
print('최소신장트리 가중치:', mst_cost)

최소신장트리: [(4, 6), (2, 5), (1, 6), (3, 5), (5, 6), (0, 1)]
최소신장트리 가중치: 25


Prim 알고리즘: 신장 트리의 정점(V)를 기반으로 동작, **O(N<sup>2</sup>)**

- 임의의 시작 정점 선택
- dist, visited 초기화
1. 선정 과정: **방문 안 한**(최소 신장 트리에 포함되지 않은) dist가 최소인(**가중치가 가장 작은**) **정점을 찾아 최소 신장 트리에 정점 추가** 후 dist 갱신
2. 적정성 검사(최소 신장 트리의 추가 후 싸이클 존재 유무 검사): X
3. 해답 점검: **최소 신장 트리의 정점 수가 N개일 때 종료**

In [14]:
import sys # for ∞

N = 7 # number of vertexes
s = 0 # start vertex
g = [None for x in range(N)] # adjacency list
g[0] = [(1, 9), (2, 10)] # g[vertex 1] = [(vertex 2, weight)]
g[1] = [(0, 9), (3, 10), (4, 5), (6, 3)]
g[2] = [(0, 10), (3, 9), (4, 7), (5, 2)]
g[3] = [(1, 10), (2, 9), (5, 4), (6, 8)]
g[4] = [(1, 5), (2, 7), (6, 1)]
g[5] = [(2, 2), (3, 4), (6, 6)]
g[6] = [(1, 3), (3, 8), (4, 1), (5, 6)]

visited = [False for x in range(N)] # in mst
dist = [sys.maxsize for x in range(N)] # distance to vertex
dist[s] = 0
previous = [None for x in range(N)] # vertex connected from (edge)
previous[s] = s

for v in range(N):
    # find vertex with smallest distance
    u = -1
    mindist = sys.maxsize
    for i in range(N):
        if not visited[i] and dist[i] < mindist:
            mindist = dist[i]
            u = i # vertex found
    visited[u] = True
    # update distance using the vertex found
    for w, wt in g[u]:
        if not visited[w]: # adjacent vertex not visited 
            if wt < dist[w]:
                dist[w] = wt
                previous[w] = u

print('최소신장트리: ', end='')
mst_cost = 0
for i in range(1, N):
    print('(%d, %d) %d' %(i, previous[i], dist[i]), end=' ')
    mst_cost += dist[i]
print('\n최소신장트리 가중치:', mst_cost)

최소신장트리: (1, 0) 9 (2, 5) 2 (3, 5) 4 (4, 6) 1 (5, 6) 6 (6, 1) 3 
최소신장트리 가중치: 25
